In [280]:
import igraph as ig
import numpy as np
from copy import deepcopy
from typing import Callable, List
import plotly.express as px
import math
import pandas as pd
import pickle
import random


In [281]:
g = ig.Graph.Read('../../data/graphs/april2022_Lspace.graphml')

In [282]:
def opercolation(graph: ig.Graph, centrality_measure: Callable[[ig.Graph], List[float]]):
    tmp_g = deepcopy(graph)  # Make a copy of the graph
    # Result list containing tuples (n of removed nodes, size of the biggest component)
    remnodes_bcsize = []
    one_percent_size = int(math.ceil(len(tmp_g.vs) / 100))
    perc_removed = 0
    rem_vertex_name = []

    while len(tmp_g.vs) >= one_percent_size:

        # Biggest component size
        bc_size = max(len(c) for c in tmp_g.components(mode='weak'))
        S = (bc_size / len(g.vs)) * 100
        average_path_length = tmp_g.average_path_length(directed=True, unconn=True) if not math.isnan(tmp_g.average_path_length(directed=True, unconn=True)) else 0

        remnodes_bcsize.append((S, perc_removed, rem_vertex_name, average_path_length))
        perc_removed += 1

        # Calculate centrality measure
        if centrality_measure is not None:
            centrality = np.array(centrality_measure(tmp_g))
            max_centrality_vertex_ids = np.argpartition(
                centrality, -one_percent_size)[-one_percent_size:]

            # Extract original nodes ids
            rem_vertex_name = [v['name'] for v in tmp_g.vs.select(max_centrality_vertex_ids)]

            # Delete vertex with max centrality measure
            tmp_g.delete_vertices(max_centrality_vertex_ids)
        # Random node removal
        else:
            random.seed(42)
            ids = random.sample(tmp_g.vs['name'], one_percent_size)
            rem_vertex_name = ids
            tmp_g.delete_vertices(ids)
    return remnodes_bcsize


In [283]:
def plot_opercolation(centrality_measure_names, *args):
    S = np.concatenate(
        [np.array([S for S, _, _, _ in x]) for x in args],
        axis=None
    )

    average_path_length = np.concatenate(
        [np.array([avg for _, _, _, avg in x]) for x in args],
        axis=None
    )

    rem_perc = np.concatenate(
        [np.array([p for _, p, _, _ in x]) for x in args],
        axis=None
    )

    removed = np.concatenate(
        [np.array([r for _, _, r, _ in x], dtype=object) for x in args],
        axis=None
    )

    centrality_measure = np.concatenate(
        [np.array([centrality_measure_names[i] for _ in x]) for i, x in enumerate(args)],
        axis=None
    )

    data = pd.DataFrame(dict(
        S=S,
        rem_perc=rem_perc,
        removed=removed,
        centrality_measure=centrality_measure
    ))

    data2 = pd.DataFrame(dict(
        average_path_length=average_path_length,
        rem_perc=rem_perc,
        removed=removed,
        centrality_measure=centrality_measure
    ))

    with open('./../../data/plots/april2022_Lspace_robustness.pickle', 'wb') as f:
        pickle.dump(data, f)

    fig = px.line(data, x="rem_perc", y="S", color="centrality_measure", line_dash="centrality_measure")
    fig.show()

    fig = px.line(data2, x="rem_perc", y="average_path_length", color="centrality_measure", line_dash="centrality_measure")
    fig.show()
    

In [284]:
random_node_op = opercolation(g, None)
degree_op = opercolation(g, lambda x: x.degree())
strength_op = opercolation(g, lambda x: x.strength(weights='num_train'))
betweennes_op = opercolation(g, lambda x: x.betweenness())
closeness_op = opercolation(g, lambda x: x.closeness()) 
clustering_op = opercolation(g, lambda x: x.transitivity_local_undirected())
pagerank_op = opercolation(g, lambda x: x.pagerank())
pagerank_numtrain_op = opercolation(g, lambda x: x.pagerank(weights='num_train'))


In [285]:
plot_opercolation(
    ['random node', 'degree', 'betweennes', 'closeness', 'clustering', 'pagerank', 'strength_num_train', 'pagerank_num_train'], 
    random_node_op, degree_op, betweennes_op, closeness_op, clustering_op, pagerank_op, strength_op, pagerank_numtrain_op
)